<p style="font-size:30px; color: #3776ab; text-align:center"><b>
Projekt 2: Epidemieausbreitung
</b></p>
<p style="font-size:25px; color: #3776ab; text-align:center">
Populationsdynamik & Netzwerke
</p>
<br>
<p style="font-size:15px; text-align:center">
von Lorenz Saalmann, Jonathan Fimmen und Peter Preisler
<br>begleitet durch Leon Sieke
</p>
<br>

# Inhaltsverzeichnis

1. Einführung
2. Verwendetes Modell
3. Grundlegende Implementation
4. Test des Grundmodells
5. Erweiterung des Modells
6. Abschließende Ergebnisse und Tests

# 1. Einführung

Das Jahr 2020 wird den meisten Menschen als das Jahr in Erinnerung bleiben, in dem die Covid-19-Pandemie den Erdball überrollt hat. Die globalisierte Welt in Verbindung mit einem passend ausgestatteten Virus haben dafür gesorgt, dass das öffentliche Leben zeitweise nahezu vollständig eingestellt werden musste um die Ausbreitung aufzuhalten. Dadurch wurde der Menschheit aufgezeigt, wie wichtig es ist die Ausbreitungsdynamik beliebiger Krankheiten zu verstehen. Denn nur dann können passende und wirksame Maßnahmen getroffen werden. Da sich unglaublich viele Faktoren auf diese Dynamik auswirken, sollten Vorbereitungen getroffen werden um möglichst gute Modelle zu erstellen. 

In dieser Arbeit werden zwei Modelle miteinander kombiniert. Die Ansteckungen werden mithilfe des SIR-Modells (Susceptible-Infected-Recovered) beschrieben, während die Reisebewegungen unter Verwendung eines Diffusionsmodells eingebracht werden. Die daraus resultierenden Differentialgleichungen werden im nächsten Kapitel diskutiert. Mit dem Modell soll anschließend die Ausbreitung einer Infektionskrankheit in Europa beschrieben werden, wobei die Flughäfen mit ihren Einzugsgebieten in der SIR-Dynamik eine Identität darstellen, welche durch das Diffusionsmodell miteiander in Verbidnung stehen.

# 2. Verwendetes Modell

2.1 SIR-Modell:

Wie der Name schon nahelegt unterscheidet das grundlegende Modell drei verschiedene Gruppen. Einmal die "Suceptible" Bevölkerung (S), welche durch die Krankheit infiziert werden kann. Dann die "Infected" Gruppe (I), welche derzeit mit der Krankheit infiziert ist. Und schließlich die "Recovered" Personen (R), welche nicht mehr von der Krankheit befallen und jetzt immun sind. Dabei wird zunächst nicht unterschieden, ob diese gestorben sind, oder sie die Krankheit besiegt haben. Es ergeben sich folgende Differntialgleichungen (wobei $N=S+I+R$ und alle zeitabhängig):

\begin{align}
    &\frac{\mathrm{d} S}{\mathrm{d} t} = -\beta \frac{S I}{N},\tag{1}\\
    &\frac{\mathrm{d} I}{\mathrm{d} t} = \beta \frac{S I}{N} -\gamma I,\tag{2}\\
    &\frac{\mathrm{d} R}{\mathrm{d} t} = \gamma I\tag{3}
\end{align}

Während der Parameter $\beta$ die Ansteckungsrate angibt, steht $\gamma$ in Verbindung zur Rate in der infizierte Personen versterben, oder sich von der Kranheit erholen. $\beta$ ist also abhängig davon, wie viele Menschen eine infizierte Person ansteckt, was widerrum von der Infektiösität des Virus' und der Zahl der Menschen, mit denen die Person Kontakt hat, abhängt. $\gamma$ ist hingegen durch die inverse durchschnittliche Dauer der Krankheit bestimmt.

2.2 Diffsionsmodell

